## Introduction to Robot Simulation (Using PyBullet)

In this notebook:
- Basic concepts for a simulation environment.

Start with imports, whereby `pybullet_data` loads built-in urdf resources.

In [1]:
import pybullet as p
import pybullet_data
import time

### Sim setup:
- Start the physics client with a window. Here with `p.connect()`, window should open.
- Add the default data path from pybullet_data.
- Apply gravity along the negative z-axis. We are dealing with physical interaction here...

In [2]:
physicsClient = p.connect(p.GUI)   # Use GUI for visualization, DIRECT for headless mode
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)

### First object
A plane is always appreciated to catch falling objects.

`plane.urdf` is part of pybullet_data.

In [3]:
planeId = p.loadURDF("plane.urdf")

### Add a box
Define the properties of the item to be added:
- size, mass, initial position, orientation

In [4]:
box_size = [0.1, 0.1, 0.1]  # units: m
mass = 1.0                  # kg
box_start_position = [0, 0, 2] 
box_start_orientation = p.getQuaternionFromEuler([0, 0, 0])

Initialize the collision and visual geometry so it can interact with its environment.

`createMultiBody()` combines the collision and visual properties into a dynamic rigid body with the given mass.

The aesthetics of the object can be set here with `p.createVisualShape`, for e.g., replacing `p.GEOM_BOX` with `p.GEOM_MESH` when using an STL or OBJ file.

In [ ]:
collision_shape = p.createCollisionShape(p.GEOM_BOX, halfExtents=box_size)
visual_shape = p.createVisualShape(p.GEOM_BOX, halfExtents=box_size, rgbaColor=[0, 0, 1, 1])
boxId = p.createMultiBody(
    baseMass=mass,
    baseCollisionShapeIndex=collision_shape,
    baseVisualShapeIndex=visual_shape,
    basePosition=box_start_position,
    baseOrientation=box_start_orientation
)

### Run the simulation for a fixed time...
- Proceed in a loop for 240 simulation steps.
- Advances the physics world at a fixed step rate.
- The box accelerates under gravity and hits the ground.
- 1./60. seconds as delay. (slow, increase to speed up..)

In [ ]:
for _ in range(240):   # ~1 second at 240 Hz
    p.stepSimulation()
    time.sleep(1./60.)

### Other possibilities...
OR run the simulatin indefinitely OR in real-time !

In [ ]:
# Comment out either here
#p.setRealTimeSimulation(1)

# or here
#print("Simulation running... Press STOP button to close. (or force quit sim window)")
#while True:
#    p.stepSimulation()
#    time.sleep(1./240.)

### Fun with the physics settings
Changing the dynamics with `p.ChangeDynamics` to make an object bouncy. (more specifically, changing the ***restitution***, other possibilities: ***lateralFricition***, ***linear/angularDamping***, etc.)

In [ ]:
import pybullet as p
import pybullet_data
import time
import os

# Setup and start the simulation window
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)

plane_id = p.loadURDF("plane.urdf")

radius = 0.1
collision_shape = p.createCollisionShape(p.GEOM_SPHERE, radius=radius)
visual_shape = p.createVisualShape(p.GEOM_SPHERE, radius=radius, rgbaColor=[1, 0, 0, 1])

start_pos = [0, 0, 2]
mass = 1.0
ball_id = p.createMultiBody(mass, collision_shape, visual_shape, start_pos)

# Set restitution ("bounciness")
p.changeDynamics(ball_id, -1, restitution=0.9)      # 0 = no bounce, 1 = very bouncy
p.changeDynamics(plane_id, -1, restitution=0.9)     # remember that bounciness has to be both side !

# --- Function to query state of the ball ---
def get_ball_state(ball_id):
    pos, orn = p.getBasePositionAndOrientation(ball_id)
    lin_vel, ang_vel = p.getBaseVelocity(ball_id)
    return {
        "position": pos,
        "orientation": orn,
        "linear_velocity": lin_vel,
        "angular_velocity": ang_vel
    }

# Run simulation in real-time
# p.setRealTimeSimulation(1)

# Run simulation in slow-mo
time_step = 1./60.
p.setRealTimeSimulation(0)
p.setTimeStep(time_step)
for step in range(1000):
    p.stepSimulation()
    state = get_ball_state(ball_id)
    print(f"Step {step}: pos={state['position']}, vel={state['linear_velocity']}", end='\r')
    time.sleep(time_step)
